In [1]:
import os
import numpy as np
import logging
import gc
import shutil
from tqdm import tqdm
import warnings
import soundfile as sf
import librosa
import subprocess
import pandas as pd
import torchaudio

warnings.filterwarnings("ignore")

In [2]:
# Configuration Constants
CONFIG = {
    'train_videos_list': r'F:\SRC_Bhuvaneswari\typpo\Crimenet\Annotations\Videolist\vtrainpadamlist.txt',
    'test_videos_list': r'F:\SRC_Bhuvaneswari\typpo\Crimenet\Annotations\Videolist\vtestpadamlist.txt',
    'train_videos_path': r'E:\SRC-Bhuvaneswari\VAD_XDViolence\ViVi\Dataset\XD Violence\Train',
    'test_videos_path': r'E:\SRC-Bhuvaneswari\VAD_XDViolence\ViVi\Dataset\XD Violence\Test',
    'train_save_path': r'E:\SRC-Bhuvaneswari\processed files\audio\ftrain',
    'test_save_path': r'E:\SRC-Bhuvaneswari\processed files\audio\ftest',
    'logs': r'F:\SRC_Bhuvaneswari\typpo\Crimenet\Utilities\logs\XDViolence_aud_first.log',
    'num_train_videos': 4264,
    'num_test_videos': 486,
    'n_train': 320,     # Limit for training videos per label
    'n_test': 80,      # Limit for testing videos per label
    'sample_rate': 16000,
    'segment_duration' : 60  # Set the desired duration (in seconds) for each audio segment
}

LABEL_MAP = {'Normal': 0, 'Abuse': 1, 'Explosion': 2, 'Fighting': 3, 'Car Accident': 4, 'Shooting': 5, 'Riot': 6}
XD_LABEL_MAP = {'A': 'Normal', 'B1': 'Fighting', 'B2': 'Shooting', 'B4': 'Riot', 'B5': 'Abuse', 'B6': 'Car Accident', 'G': 'Explosion'}

In [3]:
def configure_logging(log_path):
    if os.path.exists(log_path):
        os.remove(log_path)
    logging.basicConfig(filename=log_path, level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
    logging.info("Logging configured.")

In [4]:
def assign_labels(video_name):
    label_code = video_name.split('_label_')[1].replace('.mp4', '')
    raw_labels = label_code.split('-')
    primary_event_name = XD_LABEL_MAP.get(raw_labels[0], 'Normal')
    return LABEL_MAP.get(primary_event_name, 0)

In [5]:
def extract_audio_from_video(video_path, save_path, file_name, segment_duration):
    audio_file_path = os.path.join(save_path, f"{file_name}.wav")

    # Define the FFMPEG command to extract audio as a standard WAV
    command = [
        'ffmpeg', '-y', '-i', video_path,
        '-ac', '1',
        '-ar', str(CONFIG['sample_rate']),
        '-acodec', 'pcm_s16le',
        audio_file_path
    ]

    try:
        logging.info(f"Extracting audio from {video_path} to {audio_file_path}")
        with open(os.devnull, 'w') as fnull:
            subprocess.run(command, stdout=fnull, stderr=subprocess.STDOUT, check=True)

        if os.path.exists(audio_file_path):
            try:
                # Load the audio and segment it
                waveform, sample_rate = torchaudio.load(audio_file_path)
                segment_samples = int(segment_duration * sample_rate)
                num_segments = int(np.ceil(waveform.shape[1] / segment_samples))

                part_paths = []
                for part_num in range(num_segments):
                    start = part_num * segment_samples
                    end = min((part_num + 1) * segment_samples, waveform.shape[1])
                    segment_waveform = waveform[:, start:end]

                    # Save each segment with an updated filename
                    part_file_name = f"{file_name}_Part_{part_num+1}"
                    part_file_path = os.path.join(save_path, f"{part_file_name}.wav")
                    torchaudio.save(part_file_path, segment_waveform, sample_rate)
                    part_paths.append(part_file_path)
                    logging.info(f"Saved audio segment: {part_file_path}")

                # Clean up the full extracted audio if not needed
                os.remove(audio_file_path)

                return part_paths

            except Exception as e:
                logging.error(f"Failed to process audio segments: {e}")
                return []
        else:
            logging.error(f"Failed to create audio file at {audio_file_path}")
            return []
    
    except subprocess.CalledProcessError as e:
        logging.error(f"Error extracting audio from {video_path}: {e}")
        return []

In [6]:
def save_audio_with_labels(video_name, video_path, save_path, label, file_list, label_counters, n, segment_duration, processed_videos):
    if label == LABEL_MAP['Abuse']:
        logging.info(f"Skipping video {video_name} with label 'Abuse'")
        return
    if label_counters[label] >= n:
        return
    if video_name in processed_videos[label]:  # Skip already processed videos
        logging.info(f"Skipping duplicate video {video_name} for label {label}")
        return
    if not os.path.exists(video_path):
        logging.warning(f"Video file {video_name} does not exist, skipping.")
        return
    
    base_name = os.path.splitext(video_name)[0]
    audio_file_name = f"{base_name}_label_{label}"
    audio_segment_paths = extract_audio_from_video(video_path, save_path, audio_file_name, segment_duration)
    
    # Add all segments to the file list
    for part_num, audio_file_path in enumerate(audio_segment_paths):
        part_file_name = os.path.basename(audio_file_path)
        file_list.append((part_file_name, audio_file_path, label))
    
    # Mark the video as processed
    label_counters[label] += 1
    processed_videos[label].add(video_name)  # Track processed video

    if label_counters[label] == n:
        print(f"{list(LABEL_MAP.keys())[list(LABEL_MAP.values()).index(label)]} finish")

In [7]:
def delete_all_files(directory_path):
    try:
        for item in os.listdir(directory_path):
            item_path = os.path.join(directory_path, item)
            if os.path.isfile(item_path):
                os.remove(item_path)
                logging.info(f"Deleted file: {item}")
            elif os.path.isdir(item_path):
                shutil.rmtree(item_path)
                logging.info(f"Deleted folder: {item}")
            else:
                logging.error(f"{item} is neither a file nor a folder, skipping.")
        logging.info("All files and folders deleted successfully.")
    except Exception as e:
        logging.error(f"An error occurred while deleting files and folders: {e}")

In [8]:
def main():
    configure_logging(CONFIG['logs'])
    delete_all_files(CONFIG['train_save_path'])
    delete_all_files(CONFIG['test_save_path'])
    label_counters_train = {label: 0 for label in LABEL_MAP.values()}
    label_counters_test = {label: 0 for label in LABEL_MAP.values()}
    processed_videos_train = {label: set() for label in LABEL_MAP.values()}
    processed_videos_test = {label: set() for label in LABEL_MAP.values()}
    train_data, test_data = [], []

    with open(CONFIG['train_videos_list'], 'r') as f:
        train_video_files = [line.strip() for line in f.readlines()][:CONFIG['num_train_videos']]
    with open(CONFIG['test_videos_list'], 'r') as f:
        test_video_files = [line.strip() for line in f.readlines()][:CONFIG['num_test_videos']]

    logging.info(f"Processing {len(train_video_files)} training videos and {len(test_video_files)} testing videos.")
    
    for video_name in tqdm(train_video_files, desc="Processing train videos"):
        video_path = os.path.join(CONFIG['train_videos_path'], video_name)
        label = assign_labels(video_name)
        save_audio_with_labels(
            video_name, video_path, CONFIG['train_save_path'], label, 
            train_data, label_counters_train, CONFIG['n_train'], 
            CONFIG['segment_duration'], processed_videos_train
        )

    for video_name in tqdm(test_video_files, desc="Processing test videos"):
        video_path = os.path.join(CONFIG['test_videos_path'], video_name)
        label = assign_labels(video_name)
        save_audio_with_labels(
            video_name, video_path, CONFIG['test_save_path'], label, 
            test_data, label_counters_test, CONFIG['n_test'], 
            CONFIG['segment_duration'], processed_videos_test
        )

    train_data = [(name, path.replace('\\', '/'), label) for name, path, label in train_data]
    test_data = [(name, path.replace('\\', '/'), label) for name, path, label in test_data]

    train_df = pd.DataFrame(train_data, columns=['audio', 'path', 'label'])
    test_df = pd.DataFrame(test_data, columns=['audio', 'path', 'label'])
    train_csv_path = os.path.join(CONFIG['train_save_path'], 'train_audio_labels.csv')
    test_csv_path = os.path.join(CONFIG['test_save_path'], 'test_audio_labels.csv')

    train_df.to_csv(train_csv_path, index=False)
    test_df.to_csv(test_csv_path, index=False)
    logging.info(f"Saved Train and Test DataFrames to {train_csv_path} and {test_csv_path}")
    print("\nFinished saving Train and Test audio with labels.")

In [ ]:
if __name__ == "__main__":
    main()

Processing train videos:  13%|██████▉                                               | 545/4264 [03:14<16:54,  3.67it/s]

Normal finish


Processing train videos:  19%|██████████▏                                           | 809/4264 [03:46<06:27,  8.91it/s]